# Загрузка данных MNIST

In [14]:
import pandas as pd
import cv2


size = 28
train = pd.read_csv('mnist_train.csv', header=0, index_col=None)

cv2.imwrite('train0.png', train.iloc[0, 1:].values.reshape(size, size))
train.head()

,y.tr,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
img = cv2.imread('train0.png', cv2.IMREAD_GRAYSCALE)
img

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  

# Создание обучающей, валидационной и тестовой выборок

## Разделение на выборки

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(train.iloc[:, 1:].values, train.iloc[:, 0].values.flatten(), test_size=0.2, stratify=train['y.tr'])

del train

# нормировка
X_train = X_train.astype(np.float64) / 255
X_valid = X_valid.astype(np.float64) / 255

print(X_train, X_train.shape, '\n')
print(y_train, y_train.shape, '\n')
print(X_valid, X_valid.shape, '\n')
print(y_valid, y_valid.shape, '\n')

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (26880, 784) 

[8 4 8 ... 9 8 8] (26880,) 

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (6720, 784) 

[1 3 6 ... 6 0 8] (6720,) 



## Бинаризация целевой переменной

In [17]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(np.append(y_train, y_valid))

y_train = lb.transform(y_train)
y_valid = lb.transform(y_valid)

print(y_train, y_train.shape, '\n')
print(lb.inverse_transform(y_train))

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]] (26880, 10) 

[8 4 8 ... 9 8 8]


# Модель DNN

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.initializers import RandomNormal
from hyperopt import hp, fmin, Trials, tpe, STATUS_OK
from sklearn.metrics import accuracy_score


model = Sequential()
model.add(Dense(370, input_shape=(size ** 2,), activation='relu'))
model.add(Dense(370, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 370)               290450    
_________________________________________________________________
dense_8 (Dense)              (None, 370)               137270    
_________________________________________________________________
dense_9 (Dense)              (None, 10)                3710      
Total params: 431,430
Trainable params: 431,430
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=2048, validation_data=[X_valid, y_valid], epochs=20, verbose=1)

Train on 26880 samples, validate on 6720 samples
Epoch 1/20
26880/26880 [==============================] - 3s 114us/step - loss: 1.2287 - acc: 0.6865 - val_loss: 0.4486 - val_acc: 0.8650
Epoch 2/20
26880/26880 [==============================] - 2s 92us/step - loss: 0.3684 - acc: 0.8892 - val_loss: 0.3057 - val_acc: 0.9124
Epoch 3/20
26880/26880 [==============================] - 2s 84us/step - loss: 0.2717 - acc: 0.9200 - val_loss: 0.2617 - val_acc: 0.9253
Epoch 4/20
26880/26880 [==============================] - 2s 85us/step - loss: 0.2184 - acc: 0.9360 - val_loss: 0.2244 - val_acc: 0.9368
Epoch 5/20
26880/26880 [==============================] - 2s 85us/step - loss: 0.1857 - acc: 0.9460 - val_loss: 0.2032 - val_acc: 0.9403
Epoch 6/20
26880/26880 [==============================] - 2s 85us/step - loss: 0.1562 - acc: 0.9564 - val_loss: 0.1830 - val_acc: 0.9485
Epoch 7/20
26880/26880 [==============================] - 2s 85us/step - loss: 0.1355 - acc: 0.9617 - val_loss: 0.1750 - val_acc

# Модель CNN

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Dropout, Flatten

def make_batches(X):
    bsize = round(X.shape[1] ** 0.5)
    B = np.zeros((X.shape[0], bsize, bsize, 1), np.float64)
    for i in range(X.shape[0]):
        B[i, :, :, 0] = X[i, :].copy().reshape(bsize, bsize)
    return B

B_train = make_batches(X_train)
B_valid = make_batches(X_valid)

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=5, strides=1, activation='relu', input_shape=(size, size, 1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=3, strides=1, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(B_train, y_train, batch_size=2048, validation_data=[B_valid, y_valid], epochs=5, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)               0         
__________

KeyboardInterrupt: 